<a href="https://colab.research.google.com/github/CsonVass/raman/blob/main/DissolutionFromRaman.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# import tensorflow as tf
# tf.test.gpu_device_name()

In [3]:
# from tensorflow.python.client import device_lib
# device_lib.list_local_devices()

In [4]:
# !cat /proc/cpuinfo

In [5]:
!pip install -q keras
!pip install -q scikit-learn==0.23.1

In [6]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import tensorflow as tf
from sklearn.model_selection import train_test_split

In [7]:
# hpmcMaps = []
# dataset = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Raman/hpmc/c/DR_C01A-component-2.csv', header=None)


In [8]:
map_curve_pairs = {}
missing = []
path_to_hpmc_c = '/content/drive/MyDrive/Colab Notebooks/Raman/hpmc/c'
path_to_hpmc_v = '/content/drive/MyDrive/Colab Notebooks/Raman/hpmc/v'

paths = [path_to_hpmc_c, path_to_hpmc_v]

for path in paths:
  dir_list = os.listdir(path)
  for f in dir_list: 
    path_to_map = path + '/' + f
    map_name = f.split('-')[0].split('_')[1]
    map_curve_pairs[map_name] = {}
    map_csv = pd.read_csv(path_to_map, header=None)
    if len(map_csv.index) == 1:
      tmp = []
      for i in range(0, 31*31, 31):
        tmp.append([map_csv.iloc[0:1, i:i+31].values])
      map_csv = pd.DataFrame(tmp)
    map_curve_pairs[map_name]['map'] = map_csv

dissolutionCureves = []
curves_csv = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Raman/Tablettak_minden_adat.csv', header=None)
names = curves_csv.iloc[0:1, 1:]
time = curves_csv.iloc[4:,0:1]
curve_points = curves_csv.iloc[4:,1:].apply(lambda x: x.str.replace(',','.'))
for i in range(len(curve_points.columns)):
    curve_i = curve_points.iloc[:, i:i+1]
    if(names.iloc[0,i] in map_curve_pairs):
      map_curve_pairs[names.iloc[0,i]]['curve'] = np.asarray((curve_i).T).astype(np.float32)
    else: 
      missing.append(names.iloc[0,i])

mcp_cpy = map_curve_pairs.copy()
for name in mcp_cpy:
  if not 'curve' in map_curve_pairs[name] or len(map_curve_pairs[name]['map']) != 31:
    missing.append(map_curve_pairs[name])
    map_curve_pairs.pop(name)  

In [62]:
x = []
y = []
keys = []

for key in sorted(map_curve_pairs):
 x.append(np.asarray(map_curve_pairs[key]['map']).flatten())
 y.append(np.asarray(map_curve_pairs[key]['curve']))

x_train, x_test, y_train, y_test = train_test_split(x, y)

x_train = np.asarray(x_train)
x_train = x_train.reshape(105, -1)

y_train = np.asarray(y_train).astype(np.float32)
y_train = y_train.reshape(105, -1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  # This is added back by InteractiveShellApp.init_path()


In [63]:
from tensorflow.python.ops.nn_ops import softmax
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(128, activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(128, activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(37, activation=tf.nn.softmax))

model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits= True), 
              metrics = [tf.keras.metrics.Accuracy()])

model.fit(x_train, y_train, epochs=3)

ValueError: ignored